In [2]:
import dotenv
import os
from requests import Session
from pprint import pprint

dotenv.load_dotenv()

API_KEY = os.getenv("COINMARKETCAP_API_KEY")
API_ENDPOINT = " https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"

s = Session()

response_api = s.get(
    url=API_ENDPOINT,
    headers={"Accepts": "application/json", "X-CMC_PRO_API_KEY": API_KEY},
)

data = response_api.json()

pprint(data)

{'data': [{'circulating_supply': 19822912,
           'cmc_rank': 1,
           'date_added': '2010-07-13T00:00:00.000Z',
           'id': 1,
           'infinite_supply': False,
           'last_updated': '2025-02-11T19:32:00.000Z',
           'max_supply': 21000000,
           'name': 'Bitcoin',
           'num_market_pairs': 11931,
           'platform': None,
           'quote': {'USD': {'fully_diluted_market_cap': 1997722769053.91,
                             'last_updated': '2025-02-11T19:32:00.000Z',
                             'market_cap': 1885746792921.5217,
                             'market_cap_dominance': 60.3266,
                             'percent_change_1h': -0.64581622,
                             'percent_change_24h': -2.5375842,
                             'percent_change_30d': 0.44837373,
                             'percent_change_60d': -6.14327796,
                             'percent_change_7d': -5.36679904,
                             'percent_change_

### List and count fetched crypto 'symbols'

In [3]:
listings_symbols: set[str] = {quote.get("symbol") for quote in data.get("data", {})}

pprint(listings_symbols)
pprint(f"{len(listings_symbols)=}")

{'AAVE',
 'ADA',
 'ALGO',
 'APT',
 'ARB',
 'ATOM',
 'AVAX',
 'AXS',
 'BCH',
 'BGB',
 'BNB',
 'BONK',
 'BSV',
 'BTC',
 'BTT',
 'CRO',
 'CRV',
 'DAI',
 'DEXE',
 'DOGE',
 'DOT',
 'EGLD',
 'ENA',
 'ENS',
 'EOS',
 'ETC',
 'ETH',
 'FDUSD',
 'FET',
 'FIL',
 'FLOKI',
 'FLOW',
 'FLR',
 'FTT',
 'GALA',
 'GRT',
 'GT',
 'HBAR',
 'HNT',
 'HYPE',
 'ICP',
 'IMX',
 'INJ',
 'IOTA',
 'JASMY',
 'JTO',
 'JUP',
 'KAIA',
 'KAS',
 'KCS',
 'LDO',
 'LEO',
 'LINK',
 'LTC',
 'MELANIA',
 'MKR',
 'MNT',
 'MOVE',
 'NEAR',
 'NEO',
 'NEXO',
 'OKB',
 'OM',
 'ONDO',
 'OP',
 'PEPE',
 'POL',
 'PYTH',
 'QNT',
 'RAY',
 'RENDER',
 'RON',
 'S',
 'SAND',
 'SEI',
 'SHIB',
 'SOL',
 'SPX',
 'STX',
 'SUI',
 'TAO',
 'THETA',
 'TIA',
 'TON',
 'TRUMP',
 'TRX',
 'UNI',
 'USDC',
 'USDT',
 'USDe',
 'VET',
 'VIRTUAL',
 'WLD',
 'XAUt',
 'XCN',
 'XDC',
 'XLM',
 'XMR',
 'XRP',
 'XTZ'}
'len(listings_symbols)=100'


### Emulate v2 api output using a constant

In [9]:
COINS = "BTC,ETH"

data = response_api.json()

listings_symbols = {quote.get("symbol").upper() for quote in data.get("data", {})}

coins_select = {s.strip().upper() for s in COINS.split(",")}

if not coins_select <= listings_symbols:
    raise KeyError(f"Symbol not found: {coins_select - listings_symbols}")

quotes_select = {
    q.get("symbol"): [q] for q in data.get("data", {}) if q.get("symbol") in coins_select
}

pprint(quotes_select)

{'BTC': [{'circulating_supply': 19822912,
          'cmc_rank': 1,
          'date_added': '2010-07-13T00:00:00.000Z',
          'id': 1,
          'infinite_supply': False,
          'last_updated': '2025-02-11T19:32:00.000Z',
          'max_supply': 21000000,
          'name': 'Bitcoin',
          'num_market_pairs': 11931,
          'platform': None,
          'quote': {'USD': {'fully_diluted_market_cap': 1997722769053.91,
                            'last_updated': '2025-02-11T19:32:00.000Z',
                            'market_cap': 1885746792921.5217,
                            'market_cap_dominance': 60.3266,
                            'percent_change_1h': -0.64581622,
                            'percent_change_24h': -2.5375842,
                            'percent_change_30d': 0.44837373,
                            'percent_change_60d': -6.14327796,
                            'percent_change_7d': -5.36679904,
                            'percent_change_90d': 4.7418409,
   

### Using dataclasses
Dataclass as a single "point of truth" to refer to in other scripts.

Put in models.py

In [10]:
from dataclasses import dataclass


@dataclass
class CoinData:
    kafka_key: str
    kafka_value: dict


COINS = "BTC,ETH"

data = response_api.json()

listings_symbols = {quote.get("symbol").upper() for quote in data.get("data", {})}

coins_select = {s.strip().upper() for s in COINS.split(",")}

if not coins_select <= listings_symbols:
    raise KeyError(f"Symbol not found: {coins_select - listings_symbols}")

coin_data = [
    CoinData(
        kafka_key=q.get("symbol"),
        kafka_value=q,
    )
    for q in data.get("data", {})
    if q.get("symbol") in coins_select
]

pprint(coin_data)

kafka_keys = [c.kafka_key for c in coin_data]
pprint(kafka_keys)

[CoinData(kafka_key='BTC',
          kafka_value={'circulating_supply': 19822912,
                       'cmc_rank': 1,
                       'date_added': '2010-07-13T00:00:00.000Z',
                       'id': 1,
                       'infinite_supply': False,
                       'last_updated': '2025-02-11T19:32:00.000Z',
                       'max_supply': 21000000,
                       'name': 'Bitcoin',
                       'num_market_pairs': 11931,
                       'platform': None,
                       'quote': {'USD': {'fully_diluted_market_cap': 1997722769053.91,
                                         'last_updated': '2025-02-11T19:32:00.000Z',
                                         'market_cap': 1885746792921.5217,
                                         'market_cap_dominance': 60.3266,
                                         'percent_change_1h': -0.64581622,
                                         'percent_change_24h': -2.5375842,
                

### Dataclass for consumer

In [6]:
@dataclass
class CoinQuote:
    symbol: str
    name: str
    last_updated: str
    price: float
    percent_change_1h: float


quotes = [
    CoinQuote(
        symbol=quote.kafka_value.get("symbol", ""),
        name=quote.kafka_value.get("name", ""),
        last_updated=quote.kafka_value.get("last_updated", ""),
        price=quote.kafka_value.get("quote", {}).get("USD", {}).get("price", ""),
        percent_change_1h=quote.kafka_value.get("quote", {})
        .get("USD", {})
        .get("percent_change_1h", ""),
    )
    for quote in coin_data
]

pprint(quotes)

[CoinQuote(symbol='BTC',
           name='Bitcoin',
           last_updated='2025-02-11T19:32:00.000Z',
           price=95129.65566923375,
           percent_change_1h=-0.64581622),
 CoinQuote(symbol='ETH',
           name='Ethereum',
           last_updated='2025-02-11T19:33:00.000Z',
           price=2599.658880594776,
           percent_change_1h=-0.99739495)]
